In [6]:
import pandas as pd 
import os
from pathlib import Path
from dotenv import load_dotenv

#Local
from config import Config
from data_processing import data_processing
from utils import remove_dirs, check_and_create_dirs
from question_generator import question_generator

#Setting
pd.set_option('display.max_colwidth', None)

#CONFIG
load_dotenv()
api_key = os.getenv("PERPLEXITY_API_KEY")

cfg = Config(
    project_name="session_1",
    input_dir="data",
    api_key= api_key,
    n_questions_per_file=1,
    n_page_summary=3,
    chunk_size=5000,
    chunk_overlap=500
)

# remove_dirs(cfg) # This will delete the output directories if they exist
check_and_create_dirs(cfg)

Folder 'session_1' already exists.


In [7]:
data_processing(api_key, cfg)

File '2024-oracle-annual-report-10K.pdf' already processed. Skipping.
File '2024-cisco-full-annual-report.pdf' already processed. Skipping.
File '2024-netflix-annual-report-10K.pdf' already processed. Skipping.
File '2024-amazon-annual-report-10K.pdf' already processed. Skipping.
File '2024-meta-full-annual-report.pdf' already processed. Skipping.
File '2024-nvidia-annual-report-10K.pdf' already processed. Skipping.
File '2024-google-annual-report-10K.pdf' already processed. Skipping.
File '2024-reddit-annual-report-10K.pdf' already processed. Skipping.
File '2024-tsla-annual-report-10K.pdf' already processed. Skipping.
File '2024-apple-annual-report-10K.pdf' already processed. Skipping.


In [8]:
df = question_generator(cfg)

Generating 1 question for file: 2024-oracle-annual-report-10K.pdf. 1/10
Generating 1 question for file: 2024-cisco-full-annual-report.pdf. 2/10
Generating 1 question for file: 2024-netflix-annual-report-10K.pdf. 3/10
Generating 1 question for file: 2024-amazon-annual-report-10K.pdf. 4/10
Generating 1 question for file: 2024-meta-full-annual-report.pdf. 5/10
Generating 1 question for file: 2024-nvidia-annual-report-10K.pdf. 6/10
Generating 1 question for file: 2024-google-annual-report-10K.pdf. 7/10
Generating 1 question for file: 2024-reddit-annual-report-10K.pdf. 8/10
Generating 1 question for file: 2024-tsla-annual-report-10K.pdf. 9/10
Generating 1 question for file: 2024-apple-annual-report-10K.pdf. 10/10


In [9]:
df['question'].head()

0    Given your instructions, the provided chunk ("116/194") does not contain any substantive content for question generation, only its pagination. Since the chunk lacks useful content as required by your rules, the response is:\n\n""
1                                                                                       What is the reported aggregate market value of Cisco Systems' common stock held by non-affiliates as of January 26, 2024, in the fiscal year 2024?
2                                                                                                                        What were the total content obligations for Netflix, Inc. as of December 31, 2024, and how were they categorized?
3                                                                                                                           How did investments in Amazon's fulfillment network and technological infrastructure impact its costs in 2024?
4                                                           

# STOP

In [10]:
from user_models.example_1 import model
 
qa_chain_1 = model.model()

Loaded 1182 total documents.
Split into 1426 chunks.


KeyboardInterrupt: 

In [10]:
qa_chain_1

<module 'user_models.example_1.model' from '/Users/huybui/Desktop/SIMPLE_RAG/user_models/example_1/model.py'>

In [264]:
dummy_test = {'question': [], 'answer': [], 'top_k_chunk': []} #Top 3

for question in res['question']:
    result = qa_chain.invoke(question)
    dummy_test['question'].append(question)
    dummy_test['answer'].append(result['result'])
    dummy_test['top_k_chunk'].append(result['source_documents'])
dummy_test = pd.DataFrame(dummy_test)

In [265]:
dummy_test

,question,answer,top_k_chunk
0,What specific financial highlights or operatio...,The information provided does not directly rel...,[page_content='Oracle Financial Services Softw...
1,Based on the provided chunk of Huy Bui’s profe...,"Based on the available information, Huy Bui ha...",[page_content='•\nData and Analytics: provides...
2,"How does Cisco Systems, Inc. plan to support c...","Cisco Systems, Inc. is supporting customers in...",[page_content='6\nCisco Annual Report 2024\nAc...
3,"What provisions in Netflix, Inc.'s charter doc...","As of the fiscal year ended December 31, 2024,...",[page_content='EXHIBIT 4.18\nDESCRIPTION OF TH...
4,What are the key factors that Amazon identifie...,Amazon identified several key factors that cou...,[page_content='Guidance\nWe provided guidance ...
5,How did foreign exchange effects impact Meta P...,The search results do not provide specific inf...,[page_content='The following table presents th...
6,How does NVIDIA Corporation’s approach to leve...,NVIDIA Corporation leverages its intellectual ...,[page_content='Table of Contents\nAutomotive\n...
7,What criteria did the auditors use to evaluate...,The auditors evaluated Alphabet Inc.’s interna...,[page_content='REPORT OF INDEPENDENT REGISTERE...
8,How could changes to legal protections for con...,Changes to legal protections for content moder...,"[page_content='expenditures, and may reduce th..."
9,How did Tesla's hiring and development program...,Tesla’s hiring and development programs in 202...,[page_content='Table of Contents\nWe believe t...


In [266]:
# Combine ground truth and generated answer
combined_df = pd.merge(res, dummy_test, on='question', suffixes=('_ground_truth', '_generated'))

In [267]:
combined_df

,file_name,question,format_name,file_path,summary,chunk,chunk_id,answer,top_k_chunk
0,2024-oracle-annual-report-10K.pdf,What specific financial highlights or operatio...,2024-oracle-annual-report-10K,data/2024-oracle-annual-report-10K.pdf,The report is about Oracle Corporation and cov...,"5/25/25, 1:44 PM\n10-K\nhttps://www.sec.gov/Ar...",98,The information provided does not directly rel...,[page_content='Oracle Financial Services Softw...
1,Huy_Bui_Resume.pdf,Based on the provided chunk of Huy Bui’s profe...,Huy_Bui_Resume,data/Huy_Bui_Resume.pdf,The report does not appear to be about a speci...,Huy Bui\nwilliamhuybui@gmail.com | linkedin.co...,0,"Based on the available information, Huy Bui ha...",[page_content='•\nData and Analytics: provides...
2,2024-cisco-full-annual-report.pdf,"How does Cisco Systems, Inc. plan to support c...",2024-cisco-full-annual-report,data/2024-cisco-full-annual-report.pdf,"The report is about Cisco Systems, Inc. and co...","Cisco can help customers connect, protect and ...",22,"Cisco Systems, Inc. is supporting customers in...",[page_content='6\nCisco Annual Report 2024\nAc...
3,2024-netflix-annual-report-10K.pdf,"What provisions in Netflix, Inc.'s charter doc...",2024-netflix-annual-report-10K,data/2024-netflix-annual-report-10K.pdf,"The report is about Netflix, Inc. and covers t...",Table of Contents\nRisks Related to Our Stock ...,28,"As of the fiscal year ended December 31, 2024,...",[page_content='EXHIBIT 4.18\nDESCRIPTION OF TH...
4,2024-amazon-annual-report-10K.pdf,What are the key factors that Amazon identifie...,2024-amazon-annual-report-10K,data/2024-amazon-annual-report-10K.pdf,This report is about Amazon and covers the com...,•\nchanges in usage or adoption rates of the i...,24,Amazon identified several key factors that cou...,[page_content='Guidance\nWe provided guidance ...
5,2024-meta-full-annual-report.pdf,How did foreign exchange effects impact Meta P...,2024-meta-full-annual-report,data/2024-meta-full-annual-report.pdf,"This report is about Meta Platforms, Inc. and ...",Reconciliation of GAAP to Non-GAAP Results\n(I...,10,The search results do not provide specific inf...,[page_content='The following table presents th...
6,2024-nvidia-annual-report-10K.pdf,How does NVIDIA Corporation’s approach to leve...,2024-nvidia-annual-report-10K,data/2024-nvidia-annual-report-10K.pdf,This report is about NVIDIA Corporation and co...,"tier-1 suppliers, and start-ups. Our AV soluti...",117,NVIDIA Corporation leverages its intellectual ...,[page_content='Table of Contents\nAutomotive\n...
7,2024-google-annual-report-10K.pdf,What criteria did the auditors use to evaluate...,2024-google-annual-report-10K,data/2024-google-annual-report-10K.pdf,The report is about Alphabet Inc. and covers t...,REPORT OF INDEPENDENT REGISTERED PUBLIC ACCOUN...,72,The auditors evaluated Alphabet Inc.’s interna...,[page_content='REPORT OF INDEPENDENT REGISTERE...
8,2024-reddit-annual-report-10K.pdf,How could changes to legal protections for con...,2024-reddit-annual-report-10K,data/2024-reddit-annual-report-10K.pdf,"This report is about Reddit, Inc., a Delaware ...",Our current protections from liability for con...,55,Changes to legal protections for content moder...,"[page_content='expenditures, and may reduce th..."
9,2024-tsla-annual-report-10K.pdf,How did Tesla's hiring and development program...,2024-tsla-annual-report-10K,data/2024-tsla-annual-report-10K.pdf,The report is the Annual Report on Form 10-K f...,Table of Contents\nWe believe that sound corpo...,20,Tesla’s hiring and development programs in 202...,[page_content='Table of Contents\nWe believe t...


In [268]:
dummy_test.to_csv(f"{PROJECT_NAME}/dummy_test.csv", index=False)

# Evaluation Process

In [269]:
prompt = """
    You are a financial data Q&A evaluator.

    You are given:
    - A **question** generated from a document chunk.
    - The **document chunk** (ground truth source).
    - A **model-generated answer** to the question.

    Your job is to score the model’s answer by carefully comparing it to the document chunk.

    Use the following rubric for each category:

    ---
    **Factual Correctness**
    - 5 = All facts are fully correct and consistent with the chunk.
    - 4 = Minor factual inaccuracies but mostly correct.
    - 3 = Some factual inaccuracies, partly correct.
    - 2 = Major factual mistakes, mostly incorrect.
    - 1 = Completely factually wrong.

    ---
    **Completeness**
    - 5 = Fully answers the question with all key details.
    - 4 = Mostly complete, missing minor details.
    - 3 = Partially complete, missing important parts.
    - 2 = Mostly incomplete, only touches on part of the question.
    - 1 = Completely incomplete.

    ---
    3**Clarity**
    - 5 = Clear, precise, and easy to understand.
    - 4 = Mostly clear, with minor awkwardness.
    - 3 = Understandable but somewhat confusing or vague.
    - 2 = Hard to understand or poorly phrased.
    - 1 = Completely unclear or nonsensical.

    ---
    **Response Format**
    Return ONLY this JSON (no extra explanation):
    {
        "factual_correctness_score": [1-5],
        "completeness_score": [1-5],
        "clarity_score": [1-5],
        "comments": "A brief explanation (1-2 sentences) why you assigned these scores."
    }
"""


def evaluate_answer(question, chunk, answer):
    response = client.chat.completions.create(
        model="sonar",
        messages=[
            {"role": "system", "content": prompt},
            {
                "role": "user",
                "content": f"""
                Please evaluate the following answer based on the provided question and document chunk. 
                Return ONLY a valid JSON object.

                Question: {question}

                Document Chunk: {chunk}

                Model Answer: {answer}
                """
            }
        ],
    )

    response_content = response.choices[0].message.content.strip()
    print("LLM Raw Output:", response_content)

    # Remove duplicate keys by keeping only the last occurrence
    cleaned_content = re.sub(
        r'(,\s*")(\w+_score)":\s*\d,\s*"\2":\s*\d',
        lambda m: f',{m.group(2)}": {m.group(0).split(":")[-1]}',
        response_content
    )

    result = json.loads(cleaned_content)
    return result


In [270]:
import pandas as pd

# Prepare a list to collect all processed rows
final_rows = []

for _, row in combined_df.iterrows():
    question = row['question']
    chunk = row['chunk']
    answer = row['answer']

    success = False
    while not success:
        try:
            evaluation = evaluate_answer(question, chunk, answer)
            success = True  # Break loop if successful
        except Exception as e:
            print(f"Retrying for question: {question} due to error: {e}")

    # Build a combined result dictionary
    result_row = {
        'question': question,
        'chunk': chunk,
        'answer': answer
    }
    # Add evaluation results
    for key, value in evaluation.items():
        result_row[f'evaluation_{key}'] = value

    final_rows.append(result_row)

# Convert list of results to DataFrame
final_df = pd.DataFrame(final_rows)

# Save to CSV
final_df.to_csv('final.csv', index=False)
print("Saved final results to final.csv")


LLM Raw Output: {
  "factual_correctness_score": 5,
  "completeness_score": 4,
  "clarity_score": 5,
  "comments": "The model answer accurately reflects key financial highlights and operational changes for Oracle's fiscal year 2024, consistent with the document and referenced filings. It covers revenue growth, cloud services expansion, and operational costs, which are the main points mentioned. However, it slightly over-attributes specific details (like exact percentages) beyond the direct chunk but supported by referenced filings. Overall, the answer is clear and mostly complete but could include more explicit mention of legal and restructuring aspects noted in the filings."
}
LLM Raw Output: ```json
{
    "factual_correctness_score": 3,
    "completeness_score": 2,
    "clarity_score": 4,
    "comments": "The model's answer contains some factual inaccuracies, particularly regarding the details of Huy Bui's role and achievements at Publicis Groupe. It also lacks completeness in addres

In [271]:
final_df.head()

,question,chunk,answer,evaluation_factual_correctness_score,evaluation_completeness_score,evaluation_clarity_score,evaluation_comments
0,What specific financial highlights or operatio...,"5/25/25, 1:44 PM\n10-K\nhttps://www.sec.gov/Ar...",The information provided does not directly rel...,5,4,5,The model answer accurately reflects key finan...
1,Based on the provided chunk of Huy Bui’s profe...,Huy Bui\nwilliamhuybui@gmail.com | linkedin.co...,"Based on the available information, Huy Bui ha...",2,2,4,The model answer incorrectly states Huy Bui's ...
2,"How does Cisco Systems, Inc. plan to support c...","Cisco can help customers connect, protect and ...","Cisco Systems, Inc. is supporting customers in...",5,5,5,The answer accurately reflects the document ch...
3,"What provisions in Netflix, Inc.'s charter doc...",Table of Contents\nRisks Related to Our Stock ...,"As of the fiscal year ended December 31, 2024,...",5,5,5,The answer accurately and completely lists all...
4,What are the key factors that Amazon identifie...,•\nchanges in usage or adoption rates of the i...,Amazon identified several key factors that cou...,4,4,5,"The answer is highly accurate and clear, but i..."


In [275]:
import plotly.express as px
import pandas as pd

# Make sure relevant columns are numeric
numeric_cols = [
    'evaluation_factual_correctness_score', 
    'evaluation_completeness_score', 
    'evaluation_clarity_score'
]

df = final_df.copy()
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')  # convert invalid to NaN

# Calculate overall average score per question
df['overall_score'] = df[numeric_cols].mean(axis=1)

# Melt DataFrame to long format (including overall score)
melted_df = df.melt(
    id_vars=['question'],
    value_vars=numeric_cols + ['overall_score'],
    var_name='Metric',
    value_name='Score'
)

# Clean up metric names for display
melted_df['Metric'] = (
    melted_df['Metric']
    .str.replace('evaluation_', '', regex=False)
    .str.replace('_score', '', regex=False)
    .str.replace('_', ' ')
    .str.title()
)

# Drop rows with missing scores (optional, if needed)
melted_df = melted_df.dropna(subset=['Score'])

# Plot boxplot
fig = px.box(
    melted_df,
    x='Metric',
    y='Score',
    points='all',  # show individual points
    hover_data=['question'],
    title='Score Distributions per Metric (with Overall Score)',
    height=500
)

fig.update_layout(
    yaxis=dict(range=[0, 6], dtick=1),
    xaxis_title='Metric',
    yaxis_title='Score (1-5)'
)

fig.show()


In [276]:
import plotly.figure_factory as ff

# Select relevant columns
metrics = [
    'evaluation_factual_correctness_score',
    'evaluation_completeness_score',
    'evaluation_clarity_score',
    'overall_score'
]

# Calculate correlation matrix
corr_matrix = df[metrics].corr().round(2)

# Create heatmap
fig = ff.create_annotated_heatmap(
    z=corr_matrix.values,
    x=corr_matrix.columns.tolist(),
    y=corr_matrix.index.tolist(),
    annotation_text=corr_matrix.values,
    colorscale='Blues',
    showscale=True
)

fig.update_layout(
    title='Correlation Heatmap of Evaluation Metrics',
    width=600,
    height=600
)

fig.show()


In [277]:
#Worst answer by overall
worst_answer = df.loc[df['overall_score'].idxmin()]
print(f"Worst Answer by Overall Score: {df['overall_score'].min()}")
print(f"Question: {worst_answer['question']}")
# print(f"Answer: {worst_answer['answer']}")
print(f"Reasons: {worst_answer['evaluation_comments']}")  

Worst Answer by Overall Score: 2.3333333333333335
Question: How does NVIDIA Corporation’s approach to leveraging its intellectual property and providing GPU-based hardware solutions help automotive partners, tier-1 suppliers, and start-ups address the complex challenges in autonomous driving for fiscal year 2024?
Reasons: The model answer includes extensive details about NVIDIA's autonomous vehicle platforms and software ecosystem (DGX, Omniverse, DRIVE AGX, Halos) that are not present in the provided document chunk, which only broadly states that NVIDIA provides GPU-based hardware and licenses IP to automotive partners and tier-1 suppliers to solve complex autonomous driving challenges. The detailed references to specific products, simulation tools, safety frameworks, and ecosystem support are unsupported by the chunk. Therefore, the answer contains major factual inaccuracies relative to the source and misses the key insight from the chunk about IP licensing and collaboration. Clarity